In [ ]:
import tables_io

from rail.evaluation.dist_to_dist_evaluator import DistToDistEvaluator
from rail.evaluation.dist_to_point_evaluator import DistToPointEvaluator
from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator
from rail.core.stage import RailStage
from rail.core.data import QPHandle, TableHandle

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

# Load example Data

In [ ]:
import os
from rail.core.utils import find_rail_file
possible_local_file = './examples_data/evaluation_data/data/output_fzboost.hdf5'
if os.path.exists(possible_local_file):
    pdfs_file = os.path.abspath(possible_local_file)
else:
    pdfs_file = 'examples_data/evaluation_data/data/output_fzboost.hdf5'
    try:
        os.makedirs(os.path.dirname(pdfs_file))
    except FileExistsError:
        pass
    curl_com = f"curl -o {pdfs_file} https://portal.nersc.gov/cfs/lsst/PZ/output_fzboost.hdf5"
    os.system(curl_com)

ztrue_file = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')

In [ ]:
ensemble = DS.read_file(key='pdfs_data', handle_class=QPHandle, path=pdfs_file)
ztrue_data = DS.read_file('ztrue_data', TableHandle, ztrue_file)
truth = DS.add_data('truth', ztrue_data()['photometry'], TableHandle, path=ztrue_file)


# Dist to Dist Evaluation

In [ ]:
# 'cvm' takes about 3.5 minutes to run
# 'ad' takes about ~4 minutes to run
# 'ks' takes about 2.75 minutes to run
# 'kld' takes about X minutes to run

stage_dict = dict(
    metrics=['cvm', 'ks', 'omega', 'kld'],
    _random_state=None,
)

dtd_stage = DistToDistEvaluator.make_stage(name='SillyPoopfish', **stage_dict)

In [ ]:
dtd_results = dtd_stage.evaluate(ensemble, ensemble)

In [ ]:
results_df = tables_io.convertObj(dtd_results(), tables_io.types.PD_DATAFRAME)
results_df

# Dist to Point Evaluation

In [ ]:
stage_dict = dict(
    metrics=['cdeloss'],
    _random_state=None,
)
dtp_stage = DistToPointEvaluator.make_stage(name='SillyPoopfish', **stage_dict)

In [ ]:
dtp_results = dtp_stage.evaluate(ensemble, truth)

In [ ]:
results_df = tables_io.convertObj(dtp_results(), tables_io.types.PD_DATAFRAME)
results_df

# Point to Point Evaluation

In [ ]:
stage_dict = dict(
    metrics=['point_stats_ez'],
    _random_state=None,
)
ptp_stage = PointToPointEvaluator.make_stage(name='SillyPoopfish', **stage_dict)

In [ ]:
ptp_results = ptp_stage.evaluate(truth, truth)